In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(680, 512),
            nn.ReLU(),
            nn.Linear(512, 4),
        )

        self.decoder = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU(),
            nn.Linear(512, 680),
            nn.Sigmoid(),
        )

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = self.encoder(x)
        mu, log_var = x.chunk(2, dim=1)
        z = self.reparameterize(mu, log_var)
        return self.decoder(z), mu, log_var


def loss_function(recon_x, x, mu, log_var):
    BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction="sum")
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD


def train(model, data_loader, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (data,) in enumerate(data_loader):
        data = data.to("cuda")
        optimizer.zero_grad()
        recon_batch, mu, log_var = model(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    average_loss = train_loss / len(data_loader.dataset)
    print(f"Average loss: {average_loss:.4f}")

In [ ]:
with open("tmp/data.pkl", "rb") as f:
    data = pickle.load(f)
with open("tmp/data_id_dict.pkl", "rb") as f:
    data_id_dict = pickle.load(f)
with open("tmp/child_id_dict.pkl", "rb") as f:
    child_id_dict = pickle.load(f)
with open("tmp/word_dict.pkl", "rb") as f:
    word_dict = pickle.load(f)
with open("tmp/category_dict.pkl", "rb") as f:
    category_dict = pickle.load(f)

tensor_data = torch.tensor(data.astype(np.float32))
dataset = TensorDataset(tensor_data)
data_loader = DataLoader(dataset, batch_size=64)

model = VAE().to("cuda")
model.load_state_dict(torch.load("tmp/best_model.pth"))
# model.load_state_dict(torch.load("tmp/model_state_dict.pth"))

In [ ]:
print(data)
print(data.shape)

In [ ]:
# 単語データを潜在変数に変換
def x_to_z(model: VAE, xs: np.ndarray) -> np.ndarray:
    model.eval()
    with torch.no_grad():
        xs = torch.tensor(xs.astype(np.float32)).cuda()
        zs = model.encoder(xs)
        mu, log_var = zs.chunk(2, dim=1)
        z_points = mu.cpu()
        z_points = np.array(z_points)
        return z_points


# 潜在変数を単語データに変換
def z_to_x(model: VAE, z: np.ndarray) -> np.ndarray:
    model.eval()
    with torch.no_grad():
        z = torch.tensor(z.astype(np.float32)).cuda()
        xs = model.decoder(z)
        xs = np.array(xs.cpu())
        return xs


def get_vocabulary(xs: np.ndarray) -> np.ndarray:
    return np.sum(xs, axis=1)

In [ ]:
data_tmp = np.array([[0.5 for i in range(680)]])
data_tmp.shape

In [ ]:
def plot_origin(model, ax):
    all_0s = np.zeros((1, 680))
    z0 = x_to_z(model, all_0s)
    all_1s = np.ones((1, 680))
    z1 = x_to_z(model, all_1s)
    ax.scatter(z0[:, 0], z0[:, 1], color="blue", label="all 0s", marker="x")
    ax.scatter(z1[:, 0], z1[:, 1], color="red", label="all 1s", marker="*")
    ax.legend()


def plot_x(model, xs, ax):
    zs = x_to_z(model, xs)
    ax.scatter(zs[:, 0], zs[:, 1], s=0.2)

    plot_origin(model, ax)
    ax.set_xlabel(r"$z_{1}$")
    ax.set_ylabel(r"$z_{2}$")


def plot_x_with_vocabulary(model, data_ids, ax):
    xs = data[data_ids]
    vocabulary = get_vocabulary(xs)
    zs = x_to_z(model, xs)
    ax.scatter(zs[:, 0], zs[:, 1], c=vocabulary, cmap="turbo", s=0.2)

    plot_origin(model, ax)
    ax.set_xlabel(r"$z_{1}$")
    ax.set_ylabel(r"$z_{2}$")


def plot_x_with_age(model, data_ids, ax):
    xs = data[data_ids]
    ages = np.array([data_id_dict[i][1] for i in data_ids])
    zs = x_to_z(model, xs)
    ax.scatter(zs[:, 0], zs[:, 1], c=ages, cmap="turbo", s=0.2)

    plot_origin(model, ax)
    ax.set_xlabel(r"$z_{1}$")
    ax.set_ylabel(r"$z_{2}$")

In [ ]:
figs = {}
figs["age"] = plt.subplots()
figs["vocabulary"] = plt.subplots()

data_ids = list(data_id_dict.keys())
plot_x_with_age(model, data_ids, figs["age"][1])
plot_x_with_vocabulary(model, data_ids, figs["vocabulary"][1])

In [ ]:
# 潜在空間の格子点
import itertools


def make_lattice_points(
    z1_start: float, z1_end: float, z2_start: float, z2_end: float, spacing: float
) -> np.float32:
    z1 = np.arange(z1_start, z1_end + spacing, spacing)
    z2 = np.arange(z2_start, z2_end + spacing, spacing)

    zs = np.array(list(itertools.product(z1, z2)), dtype=np.float32)
    return zs, (len(z1), len(z2))


def plot_vocabulary(model, z1_start, z1_end, z2_start, z2_end, spacing, fig, ax):
    zs, size = make_lattice_points(z1_start, z1_end, z2_start, z2_end, spacing)
    data = z_to_x(model, zs)
    vocabulary = get_vocabulary(data).reshape(size[::-1], order="F")
    im = ax.imshow(
        vocabulary,
        extent=[z1_start, z1_end, z2_start, z2_end],
        cmap="turbo",
        origin="lower",
    )
    fig.colorbar(im, ax=ax)
    ax.set_xlabel(r"$z_{1}$")
    ax.set_ylabel(r"$z_{2}$")

In [ ]:
z1_start, z1_end = -4, 5
z2_start, z2_end = -3, 3
spacing = 0.1

zs, size = make_lattice_points(z1_start, z1_end, z2_start, z2_end, spacing)
figs["latent_vocabulary"] = plt.subplots()
plot_vocabulary(
    model,
    z1_start,
    z1_end,
    z2_start,
    z2_end,
    spacing,
    figs["latent_vocabulary"][0],
    figs["latent_vocabulary"][1],
)

In [ ]:
import random

figs["tmp"] = plt.subplots()
data_ids = []
for i, v in child_id_dict.items():
    if len(v) >= 2:
        data_ids.append([j[0] for j in v])
# print(data_ids)
n = 20
datas = random.sample(data_ids, n)
for i in datas:
    zs = x_to_z(model, data[i])
    z1, z2 = zs[:, 0], zs[:, 1]
    plt.arrow(
        z1[0],
        z2[0],
        z1[1] - z1[0],
        z2[1] - z2[0],
        head_width=0.1,
        head_length=0.1,
        fc="k",
        ec="k",
    )
# plot_vocabulary(model, z1_start, z1_end, z2_start, z2_end, spacing)
plot_x(model, data, figs["tmp"][1])
print(datas)

In [ ]:
figs["age"][0]

In [ ]:
figs["vocabulary"][0]

In [ ]:
figs["latent_vocabulary"][0]

In [ ]:
figs["tmp"][0]